### Parse Komoot tracks

In [17]:
import gpxpy
from shapely.geometry import LineString
from swak.funcflow import Pipe
import psycopg2

In [34]:
# Load and parse GPX
def load_gpx(path):
    with open(path, "r") as f:
        return gpxpy.parse(f)


def extract_linestring(gpx: gpxpy.gpx.GPX):
    points = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                if point.elevation is not None:
                    points.append((point.longitude, point.latitude, point.elevation))
    return LineString(points) 

In [40]:
l = Pipe(load_gpx, 
          extract_linestring, 
          # The linestring object is really cool and allows for all kind of things
          # Here we just want to put the wkt (well known text) into the postgres
          lambda l: l.wkt,
         )("../../resources/gps/schwemmkanalweg_gross_komoot.gpx")

In [46]:
tl

'LINESTRING Z (14.031078 48.619411 665.173207, 14.031172 48.619289 665.173207, 14.031204 48.619247 665.173207, 14.031185 48.619204 665.173207, 14.031229 48.619192 665.173207, 14.031258 48.61926 665.173207, 14.031573 48.619225 665.173207, 14.031965 48.619111 665.173207, 14.032165 48.619088 665.288316, 14.032359 48.619096 665.534886, 14.032449 48.619019 665.72161, 14.032721 48.618972 666.07824, 14.03364 48.618919 667.248439, 14.034172 48.618882 667.927017, 14.034606 48.618856 668.479812, 14.035098 48.618927 666.76639, 14.03556 48.618938 664.560357, 14.036172 48.618896 661.624278, 14.037152 48.618739 656.812471, 14.038191 48.618675 652.526442, 14.038337 48.618695 651.944503, 14.038462 48.618743 651.380295, 14.039197 48.619105 647.802985, 14.039535 48.619226 646.30278, 14.039858 48.619251 645.033523, 14.040107 48.619165 644.693355, 14.040358 48.619106 644.37846, 14.040738 48.619088 643.928126, 14.041383 48.619113 643.164389, 14.041647 48.619066 642.841207, 14.042676 48.618796 640.570565, 1

In [ ]:
### Insert into postgres

In [37]:
from geoalchemy2 import Geometry

In [39]:
Geometry(geometry_type="LINESTRINGZ", srid=4326)

Geometry(geometry_type='LINESTRINGZ', srid=4326, dimension=3, from_text='ST_GeomFromEWKT', name='geometry')

In [ ]:
# Insert into PostGIS
conn = psycopg2.connect("dbname=yourdb user=youruser password=yourpass host=localhost")
cur = conn.cursor()
cur.execute("""
    INSERT INTO hiking_tracks (name, geom)
    VALUES (%s, ST_GeomFromText(%s, 4326))
""", ("Schwemmkanalweg_groß", wkt))
conn.commit()
cur.close()
conn.close()